# 1. Install and load the necessary packages
All the packages needed from crawling to sentiment analysis can be found on this section

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import json
from urllib.request import urlopen
from pandas.io.json import json_normalize
from google_play_scraper import app,Sort, reviews
from app_store_scraper import AppStore
from pprint import pprint
import urllib3
import xmltodict
import time
from textblob import TextBlob
import spacy 
import langid 
from nltk.classify.textcat import TextCat 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text 
from sklearn.decomposition import LatentDirichletAllocation

# 2. Load data
This step can be skipped if you're scraping directly the data on the same script

In [2]:
# Loading our previously scraped data
play_store_reviews = pd.read_csv('play_store_reviews.csv', index_col=False)
app_store_reviews = pd.read_csv('app_store_reviews.csv', index_col=False)

# Add platform names to each review
app_store_reviews = app_store_reviews.assign(Platform='iOS')
play_store_reviews = play_store_reviews.assign(Platform='Android')

# Select the relevant columns
app_store_reviews = app_store_reviews[['App', 'Rating', 'Comment', 'Platform']]
play_store_reviews = play_store_reviews[['App', 'Rating', 'Comment', 'Platform']]

# Create final dataset combining reviews from App
names_ios = ['albo-reemplaza-a-tu-banco' ,'cuenca-alternativa-a-un-banco', 'klar','fondeadora-libérate-del-banco','hey-banco']
ids_playstore = ['mx.intelifin.android.albo','com.cuenca.cuenca_mobile','mx.klar.app','com.fondeadora.bank','com.banregio.hey']
names = ['albo','Cuenca', 'Klar', 'Fondeadora','hey']
for i in range(0,len(names_ios)):
    app_store_reviews['App'] = app_store_reviews['App'].str.replace(names_ios[i], names[i])
    play_store_reviews['App'] = play_store_reviews['App'].str.replace(ids_playstore[i], names[i])

df_reviews = play_store_reviews.append(app_store_reviews,ignore_index=True)

# Keep only reviews with a meaningful lenght (15 characters)
df_reviews = df_reviews[df_reviews.Comment.str.len()>=15]
df_reviews['Comment'] = df_reviews['Comment'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [3]:
df_reviews

,App,Rating,Comment,Platform
0,albo,1,Me desespera!! Tarda mucho en cargar! Cada que...,Android
1,albo,1,"Piensa dos veces antes de confiar en esta app,...",Android
2,albo,1,Al principio fue buena opcion incluso me plant...,Android
3,albo,1,No usen esta tarjeta!!! Despues de 2 semanas d...,Android
4,albo,1,que mal cada vez mas espacio ocupa y mucha act...,Android
...,...,...,...,...
43806,hey,5,"El sistema es buenisimo, sin tanto rollo, prim...",iOS
43807,hey,5,He tenido facil dos anos esta tarjeta funciona...,iOS
43808,hey,1,No me permite registrar mi INE estoy desde un ...,iOS
43809,hey,5,"Facil para usar, recibi mi tarjeta en 3 dias s...",iOS


### Classifying the comment languages.

In [4]:
# Get the language id for each review
ids_langid = df_reviews['Comment'].apply(langid.classify)

# Get just the language label
langs = ids_langid.apply(lambda tuple: tuple[0])

# Assign the language to each review
df_reviews['Language'] = langs

# How many unique language labels were applied?
print("Number of tagged languages (estimated):")
print(len(langs.unique()))

# Percent of the total dataset in English
print("Percent of data in English (estimated):")
print((sum(langs=="en")/len(langs))*100)

print("Percent of data in Spanish (estimated):")
print((sum(langs=="es")/len(langs))*100)

Number of tagged languages (estimated):
47
Percent of data in English (estimated):
0.9808840499724047
Percent of data in Spanish (estimated):
95.40665295268677


In [5]:
# 95% of the reviews are in Spanish. The population seems to be well represented in that group
#     We will select Spanish reviews only

df_reviews = df_reviews[df_reviews['Language']=='es']
df_reviews

,App,Rating,Comment,Platform,Language
0,albo,1,Me desespera!! Tarda mucho en cargar! Cada que...,Android,es
1,albo,1,"Piensa dos veces antes de confiar en esta app,...",Android,es
2,albo,1,Al principio fue buena opcion incluso me plant...,Android,es
3,albo,1,No usen esta tarjeta!!! Despues de 2 semanas d...,Android,es
4,albo,1,que mal cada vez mas espacio ocupa y mucha act...,Android,es
...,...,...,...,...,...
43806,hey,5,"El sistema es buenisimo, sin tanto rollo, prim...",iOS,es
43807,hey,5,He tenido facil dos anos esta tarjeta funciona...,iOS,es
43808,hey,1,No me permite registrar mi INE estoy desde un ...,iOS,es
43809,hey,5,"Facil para usar, recibi mi tarjeta en 3 dias s...",iOS,es


# 3. Model
Our goal is to classify bad reviews under meaningful topics

In [6]:
# What are people complaining about? Ratings below 4 and at least 15 characters
reviews = df_reviews[df_reviews['Rating']<=3]
reviews = reviews[['App','Comment']].drop_duplicates()
reviews.dropna(inplace=True)
reviews = reviews.reset_index().drop(columns='index')
print(f'% of total reviews are rated below 4: {len(reviews)/len(df_reviews)*100}')

% of total reviews are rated below 4: 18.51910283715916


In [63]:
# Create document term matrix of the reviews
#   max_df : discard words that occur more than 95% documents
#   min_df : include only those words that occur atleast in 2 documents

# Add custom stop words
my_additional_stop_words = ['app', 'aplicacion', 'banco', 'albo', 'klar', 'cuenca', 'fondeadora', 'hey','el', 'es', 'la', 'que','esta','mi','esta','dice','estaba','como','hasta','actualizacion','buena','mejor','dan',
                            'mi','una','los','pero','se','en','ya','si','lo','las','su','por','al', 'ni','era','tu','para','te','muy','mas','cuando','le','eso','ahora','solo','porque','sin','tarjeta','bien','aun','todo',
                            'asi','mis','les','del','hay','tiene','hola','ha','ver','son','mucho','100','tener','yo','tengo','pues','fue','ser','hice','nunca','nada','doy','cuenta','estrellas','da']
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

# Create document term matrix with the spanish and our custom stop-words
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
reviews_cv = cv.fit_transform(reviews['Comment'])


###### Note: each data scientist has to perform several manual combinations in order  
#          to arrive in a meaningful and interpretable set of topics and stop words

In [64]:
# LDA model with 4 topics and fit the dataset

LDA = LatentDirichletAllocation(n_components=4,random_state=1)
LDA.fit(reviews_cv)

LatentDirichletAllocation(n_components=4, random_state=1)

In [65]:
# Extract the topics and their most represented words

for index,topic in enumerate(LDA.components_):
    print(f'topic #{index} : ')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])

topic #0 : 
['sale', 'veces', 'puede', 'permite', 'datos', 'crear', 'mensaje', 'hacer', 'aparece', 'ingresar', 'credito', 'pide', 'quiero', 'registrarme', 'error', 'numero', 'deja', 'abrir', 'codigo', 'puedo']
topic #1 : 
['respuesta', 'desde', 'esperando', 'pesima', 'mes', 'transferencia', 'hacer', 'soporte', 'pesos', 'cliente', 'dicen', 'llevo', 'llega', 'pesimo', 'correo', 'atencion', 'llego', 'dias', 'dinero', 'servicio']
topic #2 : 
['pesos', 'depositar', 'recomiendo', 'depositos', 'pago', 'otras', 'transferencia', 'opcion', 'oxxo', 'tienen', 'puedes', 'sus', 'usar', 'bancos', 'transferencias', 'pagar', 'servicio', 'credito', 'hacer', 'dinero']
topic #3 : 
['funciona', 'pesima', 'antes', 'nueva', 'error', 'sesion', 'cada', 'contrasena', 'sirve', 'datos', 'problemas', 'estoy', 'acceder', 'usuario', 'ingresar', 'desde', 'hacer', 'puedo', 'entrar', 'deja']


In [66]:
# Merge the results into our initial dataset and save it
topic_reviews = LDA.transform(reviews_cv)

topics = [
'0_registration/verification',
'1_customer_support',
'2_financial_products/account_blocked',
'3_app_functionality'
]

df_topic_reviews = pd.DataFrame(topic_reviews, columns=topics)

df_result_low = pd.merge(reviews, df_topic_reviews,  how='inner', left_index=True, right_index=True)

# Add a column to the dataset with the predicted category (maximum score among our categories)
df_result_low['estimated_topic'] = df_result_low[topics].idxmax(axis = 1, skipna = True)

# Save results
df_result_low.to_csv("df_result_low.csv")

In [67]:
df_result_low

,App,Comment,0_registration/verification,1_customer_support,2_financial_products/account_blocked,3_app_functionality,estimated_topic
0,albo,Me desespera!! Tarda mucho en cargar! Cada que...,0.338127,0.175624,0.107947,0.378303,3_app_functionality
1,albo,"Piensa dos veces antes de confiar en esta app,...",0.006264,0.685027,0.302528,0.006180,1_customer_support
2,albo,Al principio fue buena opcion incluso me plant...,0.080522,0.358300,0.549391,0.011787,2_financial_products/account_blocked
3,albo,No usen esta tarjeta!!! Despues de 2 semanas d...,0.106597,0.634619,0.249942,0.008842,1_customer_support
4,albo,que mal cada vez mas espacio ocupa y mucha act...,0.009548,0.009432,0.572155,0.408865,2_financial_products/account_blocked
...,...,...,...,...,...,...,...
7038,hey,No tengo el numero con el que hice la cuenta y...,0.026986,0.025313,0.025101,0.922600,3_app_functionality
7039,hey,Desde la ultima actualizacion no puedo abrir l...,0.359063,0.050872,0.050566,0.539499,3_app_functionality
7040,hey,No recuerdo Haber visito una application mas f...,0.051167,0.053048,0.445626,0.450159,3_app_functionality
7041,hey,"No funciona la aplicacion, no te permite regis...",0.890827,0.035853,0.036327,0.036992,0_registration/verification


# 4. Validation
Let's create a random sample of 10 reviews for each app and manually check if the labels assigned are correct

In [68]:
# Create a random sample dataset and save it
seed = 2
albo = df_result_low[df_result_low['App']=='albo'].sample(n=10, random_state=seed)
klar = df_result_low[df_result_low['App']=='Klar'].sample(n=10, random_state=seed)
cuenca = df_result_low[df_result_low['App']=='Cuenca'].sample(n=10, random_state=seed)
fondeadora = df_result_low[df_result_low['App']=='Fondeadora'].sample(n=10, random_state=seed)
hey = df_result_low[df_result_low['App']=='hey'].sample(n=10, random_state=seed)
sample = albo.append(klar).append(cuenca).append(fondeadora).append(hey)

# Save results
sample.to_csv("sample.csv")
sample.head()

,App,Comment,0_registration/verification,1_customer_support,2_financial_products/account_blocked,3_app_functionality,estimated_topic
356,albo,"Pesimo servicio, los pagos no se reflejan, las...",0.018765,0.942582,0.020021,0.018632,1_customer_support
735,albo,No me ayuda en nada,0.612225,0.131288,0.125104,0.131383,0_registration/verification
1529,albo,"Algunos problemas al darme de alta, unicamente...",0.023362,0.931694,0.022025,0.022919,1_customer_support
675,albo,Todo era bueno cuando te mandaban el codigo pa...,0.870227,0.042852,0.044794,0.042127,0_registration/verification
1468,albo,ya por fin me llego el correo gracias a probar,0.047257,0.865333,0.045162,0.042248,1_customer_support


In [69]:
# Manually labeled 50 reviews (10 reviews per app) and determined if the categorisation was correct

# Load the result of the previously generated "sample.xlsx" with our manual validation input (IsAccurate)
validation = pd.read_csv('validation.csv') 
validation = validation.reset_index().drop(columns='index')
accuracy = validation['IsAccurate'].sum() / validation['IsAccurate'].count()

# Display random reviews from the validation dataset
validation.sample(n=15, random_state=1)

,Unnamed: 0,App,Comment,0_registration/verification,1_customer_support,2_financial_products/account_blocked,3_app_functionality,estimated_topic,IsAccurate
27,1808,Cuenca,No pude hacer una cuenta,0.733281,0.087425,0.088253,0.091041,0_registration/verification,1
35,3217,Fondeadora,"Falta de introducciones, no dice como cuando o...",0.597951,0.053738,0.297589,0.050722,0_registration/verification,0
40,6767,hey,Ni siquiera me deja registrarme o contratar un...,0.889943,0.036105,0.035959,0.037992,0_registration/verification,1
38,3487,Fondeadora,"Tengo una duda, puedo usar la tarjeta solo par...",0.389924,0.050810,0.506162,0.053104,2_financial_products/account_blocked,1
2,1529,albo,"Algunos problemas al darme de alta, unicamente...",0.023362,0.931694,0.022025,0.022919,1_customer_support,0
3,675,albo,Todo era bueno cuando te mandaban el codigo pa...,0.870227,0.042852,0.044794,0.042127,0_registration/verification,1
48,5227,hey,"Era algo bueno que dejo de funcionar, ya no me...",0.050565,0.444017,0.128122,0.377295,1_customer_support,1
29,1690,Cuenca,Quiero registrar mi ubicacion y dice que tiene...,0.698500,0.226845,0.036219,0.038435,0_registration/verification,1
46,4353,hey,No puedo acceder a mi cuenta no me llega el me...,0.305431,0.450193,0.031300,0.213077,1_customer_support,0
31,3228,Fondeadora,No puedo escanear mi identificacion para segui...,0.047741,0.041874,0.042291,0.868094,3_app_functionality,1


In [70]:
# Evaluation of our project
print(f'Project result: The model accuracy (number of correctly labeled reviews / total reviews) is: {accuracy *100}'+"%")

Project result: The model accuracy (number of correctly labeled reviews / total reviews) is: 76.0%


# 6. The extra mile
You may wonder what would happend if we get the topics for every different App - would reviews show a completely different scenario?

In [71]:
# How does the review distribution per app looks like? Should we get topics for each app individually?
#     Revolut represents the vast majority of the reviews, so it makes sense to explore this route

reviews['Comment'].groupby(reviews['App']).count()

App
Cuenca         453
Fondeadora     507
Klar          1226
albo          1734
hey           3123
Name: Comment, dtype: int64

In [78]:
# albo topic modelling

albo_reviews = reviews[reviews['App']=='albo']
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
reviews_cv = cv.fit_transform(albo_reviews['Comment'])

# LDA model with 4 topics
LDA = LatentDirichletAllocation(n_components=4,random_state=1)
LDA.fit(reviews_cv)

# Extract the topics and their most represented words
for index,topic in enumerate(LDA.components_):
    print(f'topic #{index} : ')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])

topic #0 : 
['internet', 'soporte', 'pide', 'codigo', 'sms', 'activar', 'servicio', 'quiero', 'cual', 'entrar', 'mensaje', 'registrarme', 'pesos', 'dicen', 'deja', 'numero', 'correo', 'hacer', 'dinero', 'puedo']
topic #1 : 
['deposito', 'mal', 'mala', 'transferencias', 'atencion', 'telefono', 'horas', 'error', 'correo', 'problema', 'transferencia', 'codigo', 'llego', 'soporte', 'puedo', 'dias', 'servicio', 'deja', 'hacer', 'dinero']
topic #2 : 
['habia', 'registrarme', 'permite', 'deja', 'problema', 'subir', 'tiempo', 'otra', 'identificacion', 'retirar', 'tomar', 'puedes', 'hacer', 'recomiendo', 'error', 'dia', 'puedo', 'foto', 'ine', 'dinero']
topic #3 : 
['desde', 'identidad', 'tiempo', 'dicen', 'llegado', 'puedo', 'despues', 'mal', '10', 'llego', 'soporte', 'llevo', 'datos', 'veces', 'pesimo', 'llega', 'codigo', 'dias', 'dinero', 'servicio']


In [74]:
# Klar topic modelling

klar_reviews = reviews[reviews['App']=='Klar']
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
reviews_cv = cv.fit_transform(klar_reviews['Comment'])

# LDA model with 4 topics
LDA = LatentDirichletAllocation(n_components=4,random_state=1)
LDA.fit(reviews_cv)

# Extract the topics and their most represented words
for index,topic in enumerate(LDA.components_):
    print(f'topic #{index} : ')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])

topic #0 : 
['informacion', 'dar', 'realizar', 'llega', 'usar', 'poder', 'pesimo', 'deja', 'puede', 'veces', 'contrasena', 'pesos', 'dia', 'correo', 'puedo', 'transferencia', 'dinero', 'servicio', 'transferencias', 'hacer']
topic #1 : 
['transferencia', 'verdad', 'dos', 'jamas', 'dicen', 'llevo', 'servicio', 'puedo', 'llega', 'semana', 'problema', 'hacer', 'problemas', 'hace', 'transferencias', 'correo', 'dias', 'desde', 'llego', 'dinero']
topic #2 : 
['aparece', 'clientes', 'tiempo', 'mes', 'han', 'hace', 'algo', 'problema', 'pesimo', 'pesima', 'mal', 'cliente', 'correo', 'dias', 'hacer', 'atencion', 'servicio', 'transferencia', 'dinero', 'puedo']
topic #3 : 
['momento', 'otra', 'pasa', 'ubicacion', 'hacer', 'error', 'servicio', 'mensaje', 'tiempo', 'registro', 'nip', 'activar', 'llega', 'dinero', 'llego', 'codigo', 'puedo', 'entrar', 'numero', 'deja']


In [75]:
# Cuenca topic modelling

cuenca_reviews = reviews[reviews['App']=='Cuenca']
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
reviews_cv = cv.fit_transform(cuenca_reviews['Comment'])

# LDA model with 4 topics
LDA = LatentDirichletAllocation(n_components=4,random_state=1)
LDA.fit(reviews_cv)

# Extract the topics and their most represented words
for index,topic in enumerate(LDA.components_):
    print(f'topic #{index} : ')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])

topic #0 : 
['sirve', 'dos', 'cuidado', 'usar', 'otra', 'llega', 'datos', 'llevo', 'soporte', 'atencion', 'han', 'dias', 'recomiendo', 'servicio', 'mala', 'depositos', 'mal', 'hacer', 'puedo', 'dinero']
topic #1 : 
['nadie', 'atencion', 'deposito', 'excelente', 'este', 'depositos', 'ellos', 'cliente', 'verdad', 'quiero', 'puedo', 'solucion', 'tienen', 'pesimo', 'problema', 'hacer', 'respuesta', 'servicio', 'whatsapp', 'dinero']
topic #2 : 
['fisica', 'problema', 'responden', 'pesimo', 'hacen', 'hacer', 'hora', 'contestan', 'problemas', 'hace', 'whatsapp', 'numero', 'vez', 'servicio', 'transferencias', 'llego', 'desde', 'otra', 'transferencia', 'dinero']
topic #3 : 
['abrir', 'acceso', 'hacer', 'transferencia', 'peor', 'fisica', 'desde', 'duda', 'transferencias', 'cobran', 'intente', 'saldo', 'aparte', 'efectivo', '10', 'depositar', 'comision', 'pesos', 'deposito', 'dinero']


In [76]:
# Fondeadora Bank topic modelling

fondeadora_reviews = reviews[reviews['App']=='Fondeadora']
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
reviews_cv = cv.fit_transform(fondeadora_reviews['Comment'])

# LDA model with 4 topics
LDA = LatentDirichletAllocation(n_components=4,random_state=1)
LDA.fit(reviews_cv)

# Extract the topics and their most represented words
for index,topic in enumerate(LDA.components_):
    print(f'topic #{index} : ')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])

topic #0 : 
['problema', 'datos', 'iniciar', 'codigo', 'sesion', 'soporte', 'correo', 'despues', 'sirve', 'quiero', 'tiempo', 'dias', 'contrasena', 'pesimo', 'dinero', 'entrar', 'servicio', 'puedo', 'deja', 'error']
topic #1 : 
['desde', 'codigo', 'error', 'cierra', 'parte', 'esto', 'hacer', 'pasa', 'llego', 'permite', 'datos', 'correo', 'transferencia', 'registrarme', 'cada', 'terminar', 'dinero', 'ahi', 'puede', 'registro']
topic #2 : 
['cada', 'pasa', 'mal', 'nuevo', 'momento', 'problema', 'funciona', 'foto', 'ine', 'ingresar', 'registro', 'cargando', 'pantalla', 'cierra', 'veces', 'crear', 'pide', 'queda', 'puedo', 'deja']
topic #3 : 
['numero', 'mismo', 'soporte', 'tienen', 'falta', 'cajeros', 'otros', 'depositar', 'oxxo', 'puedo', 'dinero', 'momento', 'usar', 'registro', 'ayuda', 'puede', 'depositos', 'efectivo', 'bancos', 'hacer']


In [77]:
# hey topic modelling

hey_reviews = reviews[reviews['App']=='hey']
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)
reviews_cv = cv.fit_transform(hey_reviews['Comment'])

# LDA model with 4 topics
LDA = LatentDirichletAllocation(n_components=4,random_state=1)
LDA.fit(reviews_cv)

# Extract the topics and their most represented words
for index,topic in enumerate(LDA.components_):
    print(f'topic #{index} : ')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-20:]])

topic #0 : 
['meses', 'mes', 'llega', 'esperando', 'cliente', 'clientes', 'segun', 'credito', 'dieron', 'pesimo', 'fisica', 'dicen', 'correo', 'llevo', 'atencion', 'llego', 'dias', 'pesos', 'dinero', 'servicio']
topic #1 : 
['deja', 'usar', 'verificacion', 'banregio', 'transferencia', 'pago', 'antes', 'sms', 'mensaje', 'codigo', 'funciona', 'puede', 'pesima', 'dejo', 'tiempo', 'credito', 'puedo', 'poder', 'opcion', 'hacer']
topic #2 : 
['problema', 'puede', 'anterior', 'permite', 'datos', 'pide', 'quiero', 'nueva', 'numero', 'contrasena', 'abrir', 'version', 'error', 'hacer', 'desde', 'usuario', 'ingresar', 'entrar', 'puedo', 'deja']
topic #3 : 
['virtual', 'atencion', 'problema', 'veces', 'mantenimiento', 'dinero', 'alta', 'abre', 'donde', 'problemas', 'deja', 'fisica', 'debito', 'servicio', 'banregio', 'mala', 'llego', 'desde', 'puedo', 'credito']


# Result
It looks like the overall topics are similar among the 4 apps.